In [1]:
import googlemaps
import requests
import os
from dotenv import load_dotenv

load_dotenv()
gmaps = googlemaps.Client(key=os.getenv("GOOGLE_KEY"))


def geocoords(place_id):
    place_details = gmaps.place(place_id)
    latitude = place_details['result']['geometry']['location']['lat']
    longitude = place_details['result']['geometry']['location']['lng']
    return {'lat': latitude, 'lon': longitude}


def get_gmaps_address(query):
    return gmaps.places_autocomplete(query, types=["geocode"], components={"country": "ca"})


def get_address_info(top_suggestion, unit_number=None):
    coords = geocoords(gmaps.find_place(top_suggestion, 'textquery')['candidates'][0]['place_id'])
    return {
        'description': top_suggestion['description'],
        'street_num': top_suggestion['terms'][0]['value'],
        'street_name': top_suggestion['terms'][1]['value'].split(' ')[0],
        'city': top_suggestion['terms'][2]['value'],
        'unit_number': unit_number,
        'lat': coords['lat'],
        'lon': coords['lon']
    }

In [15]:
def str_to_bool(s):
    if s.lower() == 'true':
         return True
    elif s.lower() == 'false':
         return False
    else:
         raise ValueError("Cannot convert to boolean: " + s)


def price_and_find_comparables(address: str, house_category: str,  unit_number:str='', beds:str='', baths:str='', house_sqft:str='', land_sqft:str='', use_comparables:str="false"):
    """
    Price a home and optionally find comparable properties.   
    :param address: The address of the property (e.g. 123 Harvie Ave Toronto, Ontario)
    :param house_category: The type of the house <<Condo, Detached, Semi, Link, Multiplex, Vacant Land, Condo Townhouse, Freehold Townhouse, Other>>
    :param unit_number: The unit number like 402 (optional)
    :param beds: The number of bedrooms (optional)
    :param baths: The number of bathrooms (optional)
    :param house_sqft: The square footage of the house (optional)
    :param land_sqft: The square footage of the land (optional)
    :param use_comparables: Whether to use comparables or not (optional). If set to true, you MUST specify at least one of beds, baths, house_sqft, land_sqft. <<true, false>>

    :return: Pricing information for the property
    """
    print(address, house_category,  unit_number, beds, baths, house_sqft, land_sqft, use_comparables)
    headers = {
        'Authorization': os.getenv("HUGGING_FACE_API_KEY"),
        'Content-Type': 'application/json'
    }
    if(str_to_bool(use_comparables) and (beds == '' and baths == '' and house_sqft == '' and land_sqft == '')):
        return 'You must specify at least one of beds, baths, house_sqft, land_sqft if you want to use comparables'
    data = [
            address,
            house_category,
            unit_number,
            str_to_bool(use_comparables),
            True,
            beds,
            baths,
            house_sqft,
            land_sqft,
            '',
            {'headers': ['None'], 'data': [['None']]}
        ]
    print(data)
    #print({'headers': ['None'], 'data': [['None']]} if modifications is None else modifications)
    response = requests.post("https://rshrott-smartbids.hf.space/run/price_with_google_maps", json={
        "data": data
    }, headers=headers)


    if response.status_code == 200:
        data = response.json()["data"]
        return str(data)
    else:
        return 'Status Code:' + str(response.status_code) + ' ' + str(response.json())
    
result = price_and_find_comparables('218 Harvie Ave Toronto','Detached', '',  '', '', '', '', "False")
print(result)

218 Harvie Ave Toronto Detached      False
['218 Harvie Ave Toronto', 'Detached', '', False, True, '', '', '', '', '', {'headers': ['None'], 'data': [['None']]}]
[{'218 Harvie Ave': {'exact_match': True, 'weight': 1, 'total_beds': 3.5, 'house-washroom': 4.0, 'house-house_area-estimate': 1917.0, 'house-land-area': 2286.0, 'prices': {'2023-06-25': {'price_complex': 1236862.6297180085, 'price_simple': 1332436.0356337286, 'px': 1236862.6297180085, 'interest_rate': 5.55249999999903}}, 'house-list_dates-date_start': '2022-05-26T00:00:00'}, 'input_info': {'description': '', 'street_num': '218', 'street_name': 'Harvie', 'city': 'Toronto', 'unit_number': '', 'lat': 43.6817537, 'lon': -79.4523143}}]
